<a href="https://colab.research.google.com/github/imabari/covid19-data/blob/master/mhlw_ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# データをダウンロード
!wget https://www.mhlw.go.jp/content/10906000/000668093.pdf -O data.pdf

--2020-09-05 01:53:50--  https://www.mhlw.go.jp/content/10906000/000668093.pdf
Resolving www.mhlw.go.jp (www.mhlw.go.jp)... 23.67.201.196, 2600:1403:7c00:19d::3194, 2600:1403:7c00:1ab::3194
Connecting to www.mhlw.go.jp (www.mhlw.go.jp)|23.67.201.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 453107 (442K) [application/pdf]
Saving to: ‘data.pdf’

data.pdf            100%[===================>] 442.49K  --.-KB/s    in 0.08s   

2020-09-05 01:53:50 (5.21 MB/s) - ‘data.pdf’ saved [453107/453107]



In [2]:
# PDFBOXをダウンロード
!wget https://www-eu.apache.org/dist/pdfbox/2.0.21/pdfbox-app-2.0.21.jar -O pdfbox-app.jar

# PDFから画像に変換
!java -jar pdfbox-app.jar PDFToImage -imageType png -dpi 200 data.pdf

--2020-09-05 01:53:50--  https://www-eu.apache.org/dist/pdfbox/2.0.21/pdfbox-app-2.0.21.jar
Resolving www-eu.apache.org (www-eu.apache.org)... 95.216.24.32, 2a01:4f9:2a:185f::2
Connecting to www-eu.apache.org (www-eu.apache.org)|95.216.24.32|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloads.apache.org/pdfbox/2.0.21/pdfbox-app-2.0.21.jar [following]
--2020-09-05 01:53:51--  https://downloads.apache.org/pdfbox/2.0.21/pdfbox-app-2.0.21.jar
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 2a01:4f8:10a:201a::2
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9897381 (9.4M) [application/java-archive]
Saving to: ‘pdfbox-app.jar’

pdfbox-app.jar      100%[===================>]   9.44M  9.62MB/s    in 1.0s    

2020-09-05 01:53:52 (9.62 MB/s) - ‘pdfbox-app.jar’ saved [9897381/9897381]



In [3]:
!add-apt-repository ppa:alex-p/tesseract-ocr -y

!apt update

!apt install tesseract-ocr
!apt install libtesseract-dev

!tesseract -v

!apt install tesseract-ocr-jpn  tesseract-ocr-jpn-vert
!apt install tesseract-ocr-script-jpan tesseract-ocr-script-jpan-vert

!tesseract --list-langs
!pip install pytesseract

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/alex-p/tesseract-ocr/ubuntu bionic InRelease [15.4 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:12 https://cloud.r-p

In [4]:
import re

import cv2
import numpy as np
import pytesseract

In [5]:
from google.colab.patches import cv2_imshow

参考：[OpenCVでの表のセルの認識方法](https://teratail.com/questions/151317)

In [6]:
img = cv2.imread("data1.png")

# BGR -> グレースケール
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

gray[gray > 220] = 255
cv2.imwrite("gray.png", gray)

# エッジ抽出 (Canny)
edges = cv2.Canny(gray, 1, 100, apertureSize=3)
cv2.imwrite("edges.png", edges)

# 膨張処理
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
edges = cv2.dilate(edges, kernel)

# 輪郭抽出
contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

In [7]:
# 面積でフィルタリング
rects = []

for cnt, hrchy in zip(contours, hierarchy[0]):
    if cv2.contourArea(cnt) < 1800:
        continue  # 面積が小さいものは除く
    if hrchy[3] == -1:
        continue  # ルートノードは除く

    # 輪郭を囲む長方形を計算する。
    rect = cv2.minAreaRect(cnt)
    rect_points = cv2.boxPoints(rect).astype(int)
    rects.append(rect_points)

In [8]:
# x-y 順でソート
rects = sorted(rects, key=lambda x: (x[0][1], x[0][0]))

data = []

for i, rect in enumerate(rects):

    # 座標
    x_max, y_max = np.amax(rect.T, axis=1)
    x_min, y_min = np.amin(rect.T, axis=1)
    # print(i, y_min, y_max, x_min, x_max)

    # 各座標ごとに切り出し
    dst = gray[y_min + 1 : y_max - 1, x_min + 5 : x_max]
    # cv2.imwrite(f"gray{i:03}.png", dst)

    txt = (
        pytesseract.image_to_string(dst, lang="jpn", config="--psm 6")
        .strip()
        .replace(",", "")
        .replace(".", "")
        .replace(" ", "")
    )
    data.append(txt)

    color = np.random.randint(0, 255, 3).tolist()
    cv2.drawContours(img, rects, i, color, 2)
    cv2.putText(img, str(i), tuple(rect[0]), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 3)

    # print("rect:\n", rect)

In [9]:
cv2.imwrite("img.png", img)

True

In [10]:
rects

[array([[ 859,  323],
        [ 859,  305],
        [1002,  305],
        [1002,  323]]), array([[178, 379],
        [178, 295],
        [370, 295],
        [370, 379]]), array([[372, 379],
        [372, 295],
        [483, 295],
        [483, 379]]), array([[488, 379],
        [488, 295],
        [605, 295],
        [605, 379]]), array([[740, 379],
        [740, 337],
        [849, 337],
        [849, 379]]), array([[ 852,  379],
        [ 852,  295],
        [1006,  295],
        [1006,  379]]), array([[1011,  379],
        [1011,  295],
        [1154,  295],
        [1154,  379]]), array([[1185,  379],
        [1185,  295],
        [1297,  295],
        [1297,  379]]), array([[607, 380],
        [607, 296],
        [850, 296],
        [850, 380]]), array([[179, 410],
        [179, 385],
        [371, 385],
        [371, 410]]), array([[372, 410],
        [372, 385],
        [483, 385],
        [483, 410]]), array([[488, 410],
        [488, 385],
        [605, 385],
        [605, 410

In [11]:
text = "\t".join(data)

In [12]:
with open("data.txt", mode="w") as f:
    f.write(text)